<a href="https://colab.research.google.com/github/hmh100298/keras_tuner/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>